In [ ]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive  
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '../..' 

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX'

import sys
from os.path import join as ospj
sys.path.append(ospj(PROJECT_ROOT, 'src'))

%load_ext autoreload
%autoreload 2

In [ ]:
""" Setup input pipeline
"""
from utils.parser import ConfigParser
import data.threeDPW as module_data 

%aimport -ConfigParser # Due to an issue of pickle and auto_reload

config = ConfigParser.wo_args(config='cfgs/project/asim-config.json', root_dir=PROJECT_ROOT)

datamodule = config.init_obj('train_loader', module_data, root_dir=PROJECT_ROOT)
train_loader = datamodule.get_loader()

datamodule = config.init_obj('validation_loader', module_data, root_dir=PROJECT_ROOT)
val_loader = datamodule.get_loader()

datamodule = config.init_obj('test_loader', module_data, root_dir=PROJECT_ROOT)
test_loader = datamodule.get_loader()


In [ ]:
""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()

In [ ]:
""" Perform test
"""

checkpoint_dir = '0604_104809' 
path = ospj(PROJECT_ROOT, f'saved/models/hppw/{checkpoint_dir}/best_val_model.pth')

trainer.load_model(path=path)

result = trainer.evaluate(loader=test_loader)

print(result)
